## Voting

Mit dem `VotingClassifier` in sklearn kann man verschiedene Klassifikatoren gleichzeitig trainieren und über die Ergebnisse aggregieren. So bekommt man oft eine höhere Genauigkeit als bei den einzelnen Klassifikationen. Nimmt man beispielsweise die Mehrheitsentscheidung, spricht man von *hard voting*.

# Random Forests

Random Forests gehören in die Familie der **Ensemble-Learner**. Hier wird nicht nur ein Klassifikator trainiert, sondern mehrere (ein **Ensemble**) und dann nach Mehrheitsentscheid ausgewählt (das funktioniert auch für die Regression).

Bevor wir konkret Random Forests trainieren, sehen wir uns ein paar Grundlagen des Ensemble Learnings an. 

In [ ]:
# Import dstools (absolute path required, please change to your systems settings)
import importlib
import sys

path = '/data/work/dstools/dstools/__init__.py'
name = 'dstools'

spec = importlib.util.spec_from_file_location(name, path)
module = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = module
spec.loader.exec_module(module)

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from dstools.datasets import titanic

In [ ]:
df = titanic()
df = df.to_numeric()
df = df.drop("PassengerId", axis=1)

In [ ]:
scaler = StandardScaler()

X = df.drop("Survived_cat", axis=1)
y = df["Survived_cat"].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33,random_state=147)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
log_clf = LogisticRegression()
rf_clf = RandomForestClassifier()
svm_clf = SVC()

In [ ]:
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf),('rf',rf_clf),('svc',svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)

In [ ]:
for clf in (log_clf, rf_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

## Bagging und Pasting

Ein anderer Ansatz als mehrere Klassifikatoren zu verwenden ist es, einen Klassifikator auf unterschiedlichen, zufällig erstellten Subsets der Trainingsdaten zu trainieren. Wird das Sample **mit** Zurücklegen erstellt, dann spricht man von **Bagging** (Abkürzung für *boostrap aggregating*), wird es **ohne** Zurücklegen erstellt nennt man es **Pasting**.

In beiden Fällen werden die Trainingsinstanzen mehrmals von dem/den Klassifikator/en (es könnten auch wieder unterschiedliche sein) verwendet. Bei Bagging kann es sogar vorkommen, dass die gleiche Trainingsinstanz mehrmals verwendet wird (weil wir ja mit Zurücklegen samplen).

Wenn alle Klassifikatoren trainiert sind, erstellt das Ensemble eine Vorhersage für einen neuen Datenpunkt, indem über alle Predictions der Klassifikatoren aggregiert wird. Dabei nimmt man üblicherweise 

- den **Modus** bei Klassifikationen
- den **Mittelwert** bei Regressionen

Jeder einzelne Prädiktor hat somit zwar einen höheren Bias, als wenn er auf den originalen Trainingsdaten trainiert wurde, insgesamt werden aber sowohl Bias als auch Varianz reduziert!

Ein weiterer Vorteil ist, dass man alle Klassifikatoren **parallel** auf mehreren CPUs rechnen kann. Diese Skalierungseigenschaft macht das Bagging sehr attraktiv

### Bagging Beispiel

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500,
    bootstrap=True,
    n_jobs=-1
)

In [ ]:
bag_clf.fit(X_train, y_train)

In [ ]:
y_pred = bag_clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

## Out-of-Bag Evaluation

Beim Bagging werden manche Instanzen der Trainingsdaten mehrmals ins Sample gewählt, manche gar nicht. Per default nimmt ein `BaggingClassifier` in sklearn $m$ Trainingsinstanzen mit Zurücklegen, wobei $m$ die Größe des Trainingssets ist. Es läßt sich zeigen, dass dadurch im Schnitt nur etwa 63% aller Trainingsinstanzen für jeden Klassifikator verwendet werden. Die verbleibenden 37%, die nicht verwendet werden, nennt man **out-of-bag** (oob)-Instanzen. Beachte, dass diese 37% für jeden Klassifikator unterschiedlich sein können.

Da der Klassifikator diese oob Insanzen während des Trainings niemals sieht, sie aber trotzdem Teil des Trainingssets sind, bietet es sich an, diese als **Validation Set** zu betrachten.

Um das durchzuführen, muss man nur die Option `oob_score=True` beim `BaggingClassifier` setzen:

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500,
    bootstrap=True,
    n_jobs=-1,
    oob_score=True
)

In [ ]:
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

Vergleichen wir das nun mit der Accuracy auf den Testdaten:

In [ ]:
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

## Random Forests in Python

Statt einen `BaggingClassifier` und einen `DecisionTreeClassifier` zu kombinieren, kann man einfacher einen `RandomForestClassifier` verwenden, der für Decision Trees optimiert wurde:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_clf = RandomForestClassifier(
    n_estimators=500,
    max_leaf_nodes=16,
    n_jobs=-1
)

In [ ]:
rf_clf.fit(X_train, y_train)

In [ ]:
y_pred_rf = rf_clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

### Erklärung

Der Random Forest hat noch ein zusätzliches Level an Randomness: anstatt nach dem besten Feature unter allen verfügbaren Features für einen Split zu suchen, betrachtet er nur ein **zufälliges** Subset an Features. Die Bäume werden so diverser, dadurch bekommt man zwar einen höheren Bias, aber eine geringere Varianz, was insgesamt zu einem besseren Modell führt!

Durch die Ausführung als Ensemble-Methode verlieren wir die Erklärbarkeit, die wir bei einzelnen Bäumen hatten. Jedoch können wir uns die **Feature-Importance** ansehen, hier am Beispiel der Iris Daten:

In [ ]:
from dstools.datasets import iris

In [ ]:
df = iris()
X = df.drop("Class", axis=1)
y = df["Class"]

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=500,n_jobs=-1)
rf_clf.fit(X, y)
for name, score in zip(list(df.columns), rf_clf.feature_importances_):
    print(name, score)